# ☀️🏃‍♀️ WeatherMate
----

**WeatherMate** is a conversational **AI agent** that analyzes real-time weather conditions and suggests the best activities and events based on location. Whether it's sunny, rainy, or snowy, WeatherMate helps you make the most of your day! 

Here's how it works:
1. Get current weather conditions for the user's location.
2. Recommend suitable indoor or outdoor activities based on the weather.
3. Find relevant events using the Ticketmaster API.
4. Merge both activity suggestions and events into a single, structured response.

---

Large Language Models (LLMs), by themselves, cannot fetch real-time data such as weather information. To enable LLMs to access and use such real-time data, we integrate **external tools.** 

In this notebook, we will implement a weather API, allowing the assistant to fetch real-time weather information and use it for personalized activity suggestions based on current weather conditions. This is an essential step in transforming an LLM into a more interactive and data-driven AI assistant.


In this notebook, we will develop a conversational AI Agent that helps users receive personalized activity recommendations based on real-time weather data.

- 🧑‍💻 Skill Level: Advanced
- 📤 Output Format: conversational chat
- 🚀 Tools:
    - Weather API integration 
    - Ticketmaster API
    - OpenAI with external tool handling
    - Gradio for the UI

🛠️ Requirements
- ⚙️ Hardware: ✅ CPU is sufficient — no GPU required
- 🔑 OpenAI API Key
- 🔑 Weather API integration (https://www.weatherapi.com)
- 🔑 Ticketmaster API (https://developer.ticketmaster.com/explore/)

⚙️ Customizable by user
- 🤖 Selected model
- 📜 system_prompt: Controls model behavior

---
📢 Find more LLM notebooks on my [GitHub repository](https://github.com/lisekarimi/lexo)

**Class Diagram**

![](https://github.com/lisekarimi/lexo/blob/main/assets/05_weather_class_diagram.png?raw=true)


In [ ]:
# imports

import os
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from datetime import datetime

# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("❌ OpenAI API Key is missing!")

weather_api_key = os.getenv('WEATHERAPI_KEY')
if not weather_api_key:
    print("❌ Weather API Key is missing!")

ticketmaster_api_key = os.getenv('TICKETMASTER_KEY')
if not ticketmaster_api_key:
    print("❌ TicketMaster API Key is missing!")


MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
# Get today's date and day name
today_str = datetime.today().strftime('%Y-%m-%d')
day_name = datetime.today().strftime('%A')

nb_activity = 10


system_message = f"""
You are a fun and helpful assistant for an Activity Suggestion App.
Your job is to recommend **up to {nb_activity} activities** based on the real-time weather fetched from the API, ensuring a mix of **indoor, outdoor, and event-based activities** whenever possible.

The total must always be **10 or fewer**, following this rule:
**nb_events + nb_indoors + nb_outdoors ≤ 10**.

You must **analyze and think carefully** to determine the best combination of activities and events for the user.
- Evaluate **weather conditions** to decide if outdoor activities are suitable.
- Check **event availability** and select the most relevant ones.
- Balance **indoor, outdoor, and event-based activities** dynamically to provide the best experience.

If one of these categories is unavailable, that's fine—just provide the best possible suggestions without exceeding **10 activities**.
Deliver everything **in one go—no waiting!**


### **Understanding Relative Dates**
- Always interpret relative dates based on **{today_str} ({day_name})**.
- The weekend always refers to Saturday and Sunday.
- "Next {day_name}" should refer to the **closest upcoming occurrence** of that day.
- If the user asks for a time range (e.g., "the next 3 days"), calculate the **exact date range** starting from today.
- If no specific date is mentioned, **assume today by default**.
- **Do not ask for confirmation** when interpreting dates—just assume the correct date and proceed confidently unless there's real ambiguity.

### **Activity and Event Suggestion Process**
To provide the best {nb_activity} activity recommendations, follow these steps:
Step 1: Retrieve Weather Data – Use the Weather API to get current conditions for the user's location.
Step 2: Suggest Activities – Recommend suitable indoor or outdoor activities based on the weather.
Step 3: Fetch Events (if available) – Use the Ticketmaster API to find relevant events in the user’s area.
Step 4: Combine Everything – Merge both event listings and activity suggestions into a single, well-structured response.
This entire process should be done seamlessly in one go without making the user wait.

### **How to Handle Each API**
- **Weather API Handling**:
    - If the user requests a relative date (e.g., "tomorrow," "next Monday"), calculate the number of days from today.
    - Provide the weather forecast only for the requested date, ignoring any other days in the response.
    - If no weather data is available, inform the user in a friendly, light-hearted way.
    - The forecast is limited to 14 days, so if the user requests a longer period, politely let him know.

- **Ticketmaster API Handling**:
    - If the user asks for events today, set the start date as today’s date.
    - If the user asks for any specific weekday, find the next occurrence of that day and use it as the start date.
    - If the user asks for a range of days (e.g., "the next 3 days"), use today’s date as the start date.
    - The country corresponding to the user's city must be represented using the ISO Alpha-2 Code (e.g., FR for France, US for the United States, CA for Canada, DK for Denmark).
    - If more than 5 events are found, ask the user for their interests to refine the search, using a one-word keyword like 'music,' 'cinema,' or 'theater.'
    - If no events are found, explicitly inform the user in a friendly, funny way.
    - Do not mention Ticketmaster unless necessary; simply state that you are checking for events.

### **User Interaction Rules**
- If the user **doesn’t mention a city**, **ask them to provide one**.
- If an event search fails, do **not** mention Ticketmaster; simply say that no events were found.
- Ensure all activity suggestions are provided **in one response**, combining weather-based activities and event suggestions.


### **Event Formatting in Output**
**If Ticketmaster events are available**, format the output as follows:
Here are some events that may interest you:
**Event Name**:
- 📅 Date: Give the date like 19th March 2025
- 📍 Venue:
- 🔗 Ticket Link: Put the URL here

(And don't forget to separate these gems with a snazzy divider)

**Event Name**:
- 📅 Date: Give the date like 19th March 2025
- 📍 Venue:
- 🔗 Ticket Link: Put the URL here

(Another divider, because we like to keep things fresh!)

**Event Name**:
- 📅 Date: Give the date like 19th March 2025
- 📍 Venue:
- 🔗 Ticket Link: Put the URL here

### **Tone and Style**
**Keep it short, fun, and don’t forget to add a dash of humor!**
Your job is to keep the user smiling while giving them the **best activities for the day**.
Be **accurate and concise**, but let’s keep it **light and lively!** 🎉
"""


In [ ]:
class WeatherAPI:
    def get_weather(self, city: str, days: int) -> dict:
        """Fetches weather data for the given city for the next 'days' number of days."""
        url = "https://api.weatherapi.com/v1/forecast.json"
        params = {"key": weather_api_key, "q": city, "days": days}
        # print(f"params weather: {params}")
        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()
            forecast = []
            for day in data["forecast"]["forecastday"]:
                forecast.append({
                    "date": day["date"],
                    "temp": day["day"]["avgtemp_c"]
                })

            result = {
                "city": city,
                "forecast": forecast
            }
            return result
        else:
            return {"error": f"City '{city}' not found or other issue. Please check the city name and try again."}

In [ ]:
from abc import ABC, abstractmethod

class BaseEventAPI(ABC):
    @abstractmethod
    def get_events(self, city, country_code, keywords, size):
        """Fetches upcoming events from an event provider."""
        pass  # Subclasses must implement this method

class TicketmasterAPI(BaseEventAPI):
    def get_events(self, city, country_code, keywords, start_date):
        """Fetches upcoming events from Ticketmaster for a given city."""
        url = "https://app.ticketmaster.com/discovery/v2/events.json"
        params = {
            "apikey": ticketmaster_api_key,
            "city": city,
            "countryCode": country_code,
            "keyword": ",".join(keywords),
            "size": 10,
            "startDateTime": start_date
        }

        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()
            events = data.get("_embedded", {}).get("events", [])
            return [
                {
                    "name": event["name"],
                    "date": event["dates"]["start"]["localDate"],
                    "venue": event["_embedded"]["venues"][0]["name"],
                    "url": event.get("url", "N/A")  # Using .get() to avoid KeyError
                }
                for event in events
            ] if events else []
        else:
            return {"error": f"API request failed! Status: {response.status_code}, Response: {response.text}"}


In [ ]:
class ChatAssistant:
    def __init__(self):
        self.model = MODEL
        self.tools = [
            {
                "type": "function",
                "function": {
                    "name": "get_weather",
                    "description": "Get the current weather and forecast for the destination city.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "city": {
                                "type": "string",
                                "description": "The city for which the weather is being requested."
                            },
                            "days": {
                                "type": "integer",
                                "description": "The number of days for the weather forecast (can be 1, 2, 6, or 10)."
                            }
                        },
                        "required": ["city", "days"],
                        "additionalProperties": False
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "get_ticketmaster_events",
                    "description": "Fetch upcoming events from Ticketmaster.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "city": {
                                "type": "string",
                                "description": "City where the events are searched."
                            },
                            "country_code": {
                                "type": "string",
                                "description": "Country code for filtering results."
                            },
                            "keywords": {
                                "type": "array",
                                "items": {
                                    "type": "string"
                                },
                                "description": "Optional keywords for event search (e.g., 'music', 'concert')."
                            },
                            "size": {
                                "type": "integer",
                                "description": "Number of events to fetch."
                            },
                            "start_date": {
                                "type": "string",
                                "description": "Start date for the event search."
                            }
                        },
                        "required": ["city", "country_code", "size", "start_date"],
                        "additionalProperties": False
                    }
                }
            }
        ]

    def chat(self, user_message, history, weather_api, event_apis):
        # Build the conversation
        messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": user_message}]

        # OpenAI response
        response = openai.chat.completions.create(model=self.model, messages=messages, tools=self.tools, stream=True)

        recovered_pieces = {
            "content": None,
            "role": "assistant",
            "tool_calls": {}
        }
        last_tool_calls = {}
        has_tool_call = False
        result = ""  # Initialize result accumulator
        # previous_index = None  # Track the last processed index

        for chunk in response:
            delta = chunk.choices[0].delta
            finish_reason = chunk.choices[0].finish_reason

            # Handle tool call detection
            if delta.tool_calls and finish_reason in [None, "tool_calls"]:
                has_tool_call = True
                piece = delta.tool_calls[0] # Get the first piece in the tool call

                # Create a dictionary for the tool call if it doesn't exist yet
                recovered_pieces["tool_calls"][piece.index] = recovered_pieces["tool_calls"].get(
                    piece.index, {"id": None, "function": {"arguments": "", "name": ""}, "type": "function"}
                )

                if piece.id:
                    recovered_pieces["tool_calls"][piece.index]["id"] = piece.id
                if piece.function.name:
                    recovered_pieces["tool_calls"][piece.index]["function"]["name"] = piece.function.name
                recovered_pieces["tool_calls"][piece.index]["function"]["arguments"] += piece.function.arguments

                # Store the tool call in the dictionary by index
                last_tool_calls[piece.index] = recovered_pieces["tool_calls"][piece.index]

            # Store content in result and yield
            else:
                result += delta.content or ""
                if result.strip():
                    yield result


        # Handle tool call scenario
        if has_tool_call:
            # Handle the tool calls
            response = self.handle_tool_call(last_tool_calls, weather_api, event_apis)

            if response:  # Only iterate if response is not None
                tool_calls_list = [tool_call for tool_call in last_tool_calls.values()]
                messages.append({"role": "assistant", "tool_calls": tool_calls_list}) # Append the tool calls to the messages

                # Dynamically process each tool call response and append it to the message history
                for res in response:
                    messages.append({
                        "role": "tool",
                        "tool_call_id": res["tool_call_id"],
                        "content": json.dumps(res["content"])
                    })

            # New OpenAI request with tool response
            response = openai.chat.completions.create(model=self.model, messages=messages, stream=True)

            result = ""  # Reset result before second stream
            for chunk in response:
                result += chunk.choices[0].delta.content or ""
                if result.strip():
                    yield result


    def handle_tool_call(self, tool_call, weather_api, event_apis):
        stored_values = {}  # Dictionary to store the valid value for each field

        for index, call in tool_call.items():
            # Load the arguments for each tool call dynamically
            arguments = json.loads(call["function"]["arguments"])

            # Iterate over all keys dynamically
            for key, value in arguments.items():
                # Update the field if it's currently None or hasn't been set before
                if key not in stored_values or stored_values[key] is None:
                    stored_values[key] = value

        city = stored_values.get('city')
        days = stored_values.get('days')
        country_code = stored_values.get('country_code')
        keywords = stored_values.get('keywords', [])
        # size = stored_values.get('size')
        start_date = stored_values.get('start_date')
        start_date = str(start_date) + "T00:00:00Z"

        weather_data = None
        event_data = None

        # Iteration over tool_call
        for call in tool_call.values():
            if call["function"]["name"] == "get_weather":
                weather_data = weather_api.get_weather(city, days)

            if call["function"]["name"] == "get_ticketmaster_events":
                event_data = event_apis["ticketmaster"].get_events(city, country_code, keywords, start_date)

        responses = []

        # Ensure weather response is always included
        weather_tool_call_id = next((call["id"] for call in tool_call.values() if call["function"]["name"] == "get_weather"), None)
        if weather_data and "forecast" in weather_data:
            responses.append({
                "role": "assistant",
                "content": {"weather": weather_data["forecast"]},
                "tool_call_id": weather_tool_call_id
            })
        elif weather_tool_call_id:
            responses.append({
                "role": "assistant",
                "content": {"message": "No weather data available for this location."},
                "tool_call_id": weather_tool_call_id
            })

        # Ensure event response is always included
        event_tool_call_id = next((call["id"] for call in tool_call.values() if call["function"]["name"] == "get_ticketmaster_events"), None)
        if event_data:
            responses.append({
                "role": "assistant",
                "content": {"events": event_data},
                "tool_call_id": event_tool_call_id
            })
        elif event_tool_call_id:
            responses.append({
                "role": "assistant",
                "content": {"message": "No events found for this location."},
                "tool_call_id": event_tool_call_id
            })

        # print("Final responses:", responses)
        return responses


In [ ]:
# GradioInterface class to handle the Gradio UI
class GradioInterface:
    def __init__(self, activity_assistant):
        self.activity_assistant = activity_assistant

    def launch(self):
        # Gradio chat interface
        gr.ChatInterface(fn=self.activity_assistant.chat, type="messages").launch()

# ActivityAssistant setup
class ActivityAssistant:
    def __init__(self):
        self.weather_api = WeatherAPI()  # Interact with the Weather API
        self.event_apis = { # Interact with the Events API
            "ticketmaster": TicketmasterAPI()
        }
        self.chat_assistant = ChatAssistant()  # This will handle conversation with OpenAI

    def chat(self, user_message, history):
        # Forward the user message and conversation history to ChatAssistant
        response_stream =  self.chat_assistant.chat(user_message, history, self.weather_api, self.event_apis)
        for chunk in response_stream:
            yield chunk

In [ ]:
# Main execution
if __name__ == "__main__":
    activity_assistant = ActivityAssistant()
    gradio_interface = GradioInterface(activity_assistant)
    gradio_interface.launch()